## SVM Experiments

Code Model selection code

## Imports

In [1]:
##### imports ############
import pandas as pd
import numpy as np
import json

import joblib

import os
import math

import torch
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, XLNetTokenizer, XLNetModel, XLNetLMHeadModel, XLNetConfig
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange
import matplotlib.pyplot as plt

from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn

from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score


/home/wkhal001/envs/Twitter_crisis/lib/python3.7/site-packages/_distutils_hack/__init__.py:19: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  "Distutils was imported before Setuptools. This usage is discouraged "


In [2]:
#from utils_crisis_classification import clean_text
from preprocessing_step import clean_text

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/wkhal001/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from string import punctuation
from os import listdir
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

import pandas as pd
import numpy as np
import collections
from collections import Counter

from sklearn.model_selection import train_test_split
import string

import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))

import pandas as pd
import numpy as pn
from numpy import mean

from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split

import string

import collections
from collections import Counter

from string import punctuation
from os import listdir
from numpy import array

from pickle import load
from numpy import array

import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))

import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate


from keras.layers import LSTM
from sklearn.decomposition import PCA

import pickle
import numpy as np
from numpy import mean
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
import statistics as s

import matplotlib.pyplot as plt


import sys

/home/wkhal001/envs/Twitter_crisis/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:68: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.4.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [4]:
##### imports ############
import pandas as pd
import numpy as np
import json

import joblib

import os
import math

import torch
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, XLNetTokenizer, XLNetModel, XLNetLMHeadModel, XLNetConfig
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange
import matplotlib.pyplot as plt

from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn

from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

#from utils_crisis_classification import clean_text

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

from transformers import AdamW
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast

# specify GPU
device = torch.device("cuda")

from Baseline_Models import Display_metrics,Display_classification_report,Confusion_matrix
from keras.backend import clear_session

## Read Data Set

In [5]:
## Read Harvey data set ##
labeledDF=pd.read_csv("/home/wkhal001/Desktop/data_rescue_mining/labeled_ds_Corrected_csv_updated_22_09_13.csv") 
del labeledDF['Unnamed: 0']

In [6]:
labeledDF.columns

Index(['id', 'status_id', 'created_at', 'text', 'address', 'loc', 'situ',
       'save', 'sos', 'sos.pred', 'sos.correct'],
      dtype='object')

In [7]:
labeledDF = labeledDF[['id','created_at','text','sos.correct']]

labeledDF['cleaned_tweet'] = labeledDF['text'].apply(lambda x: " ".join(clean_text(x)))

In [11]:
## Extract useful columns 
df_training = labeledDF[['id','text','cleaned_tweet','sos.correct']]
df_training.columns= ['id','non_cleaned_text','text','label']

In [13]:
df_training['label'].value_counts()

0    5520
1     272
Name: label, dtype: int64

## SVM model selection

Find the best SVM model on the training set using the repeated K-fold cross validation process

TF-IDF schemes: TF-IDF1 500 unigrams/bigrams -- TF-IDF2 500 unigrams -- TF-IDF3 1000 unigrams/bigrams -- TF-IDF4 1000 unigrams

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

from Baseline_Models import Create_TFIDF,Vector_Encoding_TFIDF,Create_BOW,Vector_Encoding_BOW

TFID = Create_TFIDF(df_training['text'],500,1,1)


rep = 0
save_sets = {}
for i in range(5):
    
    #Train_Y.value_counts()
    skf = StratifiedKFold(n_splits=10,shuffle=True,random_state=rep)
    
    ###### SVM - vectorize input text using TF-IDF ######
    Train_X_TFIDF = Vector_Encoding_TFIDF(TFID,df_training['text'])
    
    ###### defining parameter range ######
    param_grid = {'C': [1,5,10,15,20,50,100,150],  
              'gamma': [0.001,0.1,1,5,10],  
              'degree' : [2,3],
              'kernel': ['linear','poly','rbf']} 
   
    ###### define grid search object ################
    grid = GridSearchCV(SVC(),param_grid, cv = skf, refit = True, scoring = 'f1' ,verbose = 3,n_jobs=-1)   #cv = rkf   #scoring = 'f1'   #verbose = 3
    
    ###### fitting the model for grid search ################
    grid.fit(Train_X_TFIDF, Train_Y)
    
    ###### save data and results for the current split ################
    save_sets.update({rep:{'grid':grid}})
    rep = rep + 1 


Fitting 5 folds for each of 240 candidates, totalling 1200 fits


In [44]:
######### save grids ##########
ls_grid = [save_sets[0]['grid'],save_sets[1]['grid'],save_sets[2]['grid'],save_sets[3]['grid'],save_sets[4]['grid']]
path = "/home/wkhal001/Crisis-classification/scripts-rescue-detection/___rescue_identification_experiments/SVM_experiments/SVM_CV_IAN_IDA"

iteration =1
for g in ls_grid:  
    save_path = path + "/" + "grid-CV-iter"+ str(iteration)
    joblib.dump(g, save_path)
    iteration = iteration + 1

In [45]:
######### load grids ##########
import joblib

### save repetitions grids
grid1 = joblib.load("/home/wkhal001/Crisis-classification/scripts-rescue-detection/___rescue_identification_experiments/SVM_experiments/SVM_CV/grid-CV-iter1")
grid2 = joblib.load("/home/wkhal001/Crisis-classification/scripts-rescue-detection/___rescue_identification_experiments/SVM_experiments/SVM_CV/grid-CV-iter2")
grid3 = joblib.load("/home/wkhal001/Crisis-classification/scripts-rescue-detection/___rescue_identification_experiments/SVM_experiments/SVM_CV/grid-CV-iter3")
grid4 = joblib.load("/home/wkhal001/Crisis-classification/scripts-rescue-detection/___rescue_identification_experiments/SVM_experiments/SVM_CV/grid-CV-iter4")
grid5 = joblib.load("/home/wkhal001/Crisis-classification/scripts-rescue-detection/___rescue_identification_experiments/SVM_experiments/SVM_CV/grid-CV-iter5")

ls_grid = [grid1,grid2,grid3,grid4,grid5]

In [46]:
### best models and scores
best_models = [g.best_params_ for g in ls_grid]
best_f_scores = [g.best_score_ for g in ls_grid]

In [47]:
## best models in 5 reps
best_models

[{'C': 20, 'degree': 2, 'gamma': 0.1, 'kernel': 'rbf'},
 {'C': 150, 'degree': 2, 'gamma': 0.1, 'kernel': 'rbf'},
 {'C': 100, 'degree': 2, 'gamma': 0.1, 'kernel': 'rbf'},
 {'C': 20, 'degree': 2, 'gamma': 0.1, 'kernel': 'rbf'},
 {'C': 15, 'degree': 2, 'gamma': 0.1, 'kernel': 'rbf'}]

In [48]:
## best scores in 5 reps
best_f_scores

[0.8376376169749038,
 0.833931786859323,
 0.834361789513754,
 0.8302264750947363,
 0.8348285178176182]